In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import shapely
from shapely import Polygon, LineString

In [ ]:
img_path = r"data\sample.jpg"
img_color = cv2.imdecode(np.fromfile(img_path, dtype=np.uint8), 1)
# img_color = cv2.rotate(img_color, cv2.ROTATE_90_COUNTERCLOCKWISE)

img = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
img = cv2.GaussianBlur(img, (7, 7), 1, 1)
print(img.shape)

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(9, 9))
img = clahe.apply(img)

_, img_binary = cv2.threshold(img, thresh=0, maxval=255, type=cv2.THRESH_BINARY|cv2.THRESH_OTSU)

plt.imshow(img, cmap='gray')
plt.figure(2)
plt.imshow(img_binary, cmap='gray')


In [ ]:
contours, hierarchy = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour_area_max = 0
i_max = 0
for i, contour in enumerate(contours):
    contour_area = cv2.contourArea(contour)
    if contour_area > contour_area_max:
        contour_area_max = contour_area
        i_max = i

contour = np.array(contours[i_max]).squeeze()
print(contour.shape)

poly_contour = Polygon(contour)
cx = int(poly_contour.centroid.x)
cy = int(poly_contour.centroid.y)
print("center point: ", cx, cy)

# 将轮廓点按照四个角分为四组
contour_lt = contour[np.logical_and(contour[:, 0] < cx, contour[:, 1] < cy)]
dist_lt = np.sqrt(np.power(cx-contour_lt[:, 0], 2) + np.power(cy-contour_lt[:, 1], 2))
max_lt = contour_lt[np.argmax(dist_lt)]

contour_rt = contour[np.logical_and(contour[:, 0] >= cx, contour[:, 1] < cy)]
dist_rt = np.sqrt(np.power(cx-contour_rt[:, 0], 2) + np.power(cy-contour_rt[:, 1], 2))
max_rt = contour_rt[np.argmax(dist_rt)]

contour_lb = contour[np.logical_and(contour[:, 0] < cx, contour[:, 1] >= cy)]
dist_lb = np.sqrt(np.power(cx-contour_lb[:, 0], 2) + np.power(cy-contour_lb[:, 1], 2))
max_lb = contour_lb[np.argmax(dist_lb)]

contour_rb = contour[np.logical_and(contour[:, 0] >= cx, contour[:, 1] >= cy)]
dist_rb = np.sqrt(np.power(cx-contour_rb[:, 0], 2) + np.power(cy-contour_rb[:, 1], 2))
max_rb = contour_rb[np.argmax(dist_rb)]

# 绘制轮廓
contour_mask = np.zeros(img.shape, dtype=np.uint8)
cv2.fillPoly(contour_mask, [contour.reshape(-1,1,2).astype(np.int32)], (255,0,0))

# contour_image = cv2.drawContours(contour_mask, contours, i_max, (255, 255, 0), 2)
# cv2.circle(contour_image, max_lt, 10, (255,0,0), -1)
# cv2.circle(contour_image, max_rt, 10, (255,0,0), -1)
# cv2.circle(contour_image, max_lb, 10, (255,0,0), -1)
# cv2.circle(contour_image, max_rb, 10, (255,0,0), -1)

# cv2.line(contour_image, max_lt, max_rt, (255,0,0), 10)
# cv2.line(contour_image, max_rt, max_rb, (255,0,0), 10)
# cv2.line(contour_image, max_rb, max_lb, (255,0,0), 10)
# cv2.line(contour_image, max_lb, max_lt, (255,0,0), 10)

plt.imshow(contour_mask, cmap='gray')

In [ ]:
content_w = (max_rt[0] -  max_lt[0]) // 2
content_h = (max_lb[1] - max_lt[1]) // 2
print(content_w, content_h)

dst_lt = (cx-content_w, cy-content_h)
dst_rt = (cx+content_w, cy-content_h)
dst_lb = (cx-content_w, cy+content_h)
dst_rb = (cx+content_w, cy+content_h)

ori_point_set = np.array([max_lt, max_rt, max_lb, max_rb]).astype(np.float32)
dst_point_set = np.array([dst_lt, dst_rt, dst_lb, dst_rb]).astype(np.float32)
trans_matrix = cv2.getPerspectiveTransform(ori_point_set, dst_point_set)
# trans_matrix, _ = cv2.findHomography(ori_point_set, dst_point_set, 0)

img_color[contour_mask==0] = (255, 255, 255)

dst_img = cv2.warpPerspective(img_color, trans_matrix, img.shape[::-1], borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

save_path = r""
cv2.imencode(".jpg", dst_img)[1].tofile(save_path)

plt.imshow(cv2.cvtColor(dst_img, cv2.COLOR_BGR2RGB), cmap='gray')